# Hate Speech Detector - PL - Features extraction for Conv. & LSTM model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import csv
import pandas as pd
import numpy as np
import pickle
from klepto.archives import dir_archive

import sys
import string
import re
import fasttext

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'lstm'

## Poleval 2019 data loading

### Classes pre:
    0 - non-harmful
    1 - cyberbullying
    2 - hate speech

In [3]:
if not os.path.exists('hsd/Poleval2019/perfect_data.pkl'):
    with open('hsd/Poleval2019/train_texts.txt', 'r') as f:
        tweets = f.readlines()
    with open('hsd/Poleval2019/test_texts.txt', 'r') as f:
        tweets.extend(f.readlines())
    
    with open('hsd/Poleval2019/train_labels.txt', 'r') as f:
        labels = f.readlines()
    with open('hsd/Poleval2019/test_labels.txt', 'r') as f:
        labels.extend(f.readlines())
    
    with open('hsd/Poleval2019/perfect_data.pkl', 'w') as f:
        def chcl(c):
            return 0 if c=='0\r\n' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/Poleval2019/perfect_data.pkl', 'r') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
list(zip(tweets[:5], labels[:5]))

[('Dla mnie faworytem do tytu\xc5\x82u b\xc4\x99dzie Cracovia. Zobaczymy, czy typ si\xc4\x99 sprawdzi.\r\n',
  0),
 ('@anonymized_account @anonymized_account Brawo ty Daria kibic ma by\xc4\x87 na dobre i z\xc5\x82e\r\n',
  0),
 ('@anonymized_account @anonymized_account Super, polski premier sk\xc5\x82ada kwiaty na grobach kolaborant\xc3\xb3w. Ale doczekali\xc5\x9bmy czas\xc3\xb3w.\r\n',
  0),
 ('@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.\r\n',
  0),
 ('Odrzut natychmiastowy, kwa\xc5\x9bna mina, mam problem\r\n', 0)]

## Features extraction

In [5]:
def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = re.sub(hashtag_regex, '', parsed_text)
    return parsed_text

def pos(text):
    import morfeusz2
    morf = morfeusz2.Morfeusz()

    analysis = morf.analyse(line)
    
    return [interp[2] for i, j, interp in analysis]
    

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['PUSTY']*additional

In [6]:
# if no morfeusz2 installed then save preprocessed tweets and load pos strings from outer source
sentences = [preprocess(t) for t in tweets]
with open('hsd/Poleval2019/preprocessed.pkl', 'w') as f:
    pickle.dump(sentences, f)

### Median sentences length

In [7]:
def median_sentences_length(data):
    all_lengths, wt_lengths, pos_lengths = [], [], []
    for d in tqdm(data):
        sentence = preprocess(d)
        all_lengths.append(len(sentence.split(' ')))
        wt_lengths.append(len(sentence.split(' ')))
        
        # only if morfeusz2 is installed
        '''all_lengths.append(len(pos(preprocess(d))))
        pos_lengths.append(len(pos(preprocess(d))))'''
    # otherwise load pos strings from outer source
    with open('hsd/Poleval2019/pos_sentences.pkl', 'r') as f:
        pos_sentences = pickle.load(f)
    for ps in tqdm(pos_sentences):
        all_lengths.append(len(ps.split(' ')))
        pos_lengths.append(len(ps.split(' ')))
    
    return int(np.median(all_lengths)), int(np.median(wt_lengths)), int(np.median(pos_lengths))

In [8]:
opt_length, opt_wt_length, opt_pos_length = median_sentences_length(tweets)
dim = 6*20 if MODEL == 'conv' else 200

print('Optimal all length: {}'.format(opt_length))
print('Optimal sentence length: {}'.format(opt_wt_length))
print('Optimal pos sentence length: {}'.format(opt_pos_length))


Optimal all length: 17
Optimal sentence length: 13
Optimal pos sentence length: 27


### Supervised fastText wordtokens training

In [9]:
if not os.path.exists('hsd/Poleval2019/fasttext.ft'):
    with open('hsd/Poleval2019/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)))

# load fasttext model or train & save if none
if os.path.exists('hsd/Poleval2019/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/Poleval2019/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/Poleval2019/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/Poleval2019/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [10]:
def get_wordtoken_fts(data, length):
    
    sentences_words = []
    for d in tqdm(data):
        sentence = preprocess(d)
        sentences_words.append(sentence.split(' '))
    
    sentences_words = [pad_words(sw, length) for sw in sentences_words]
    
    ft_matrices = []
    for sw in tqdm(sentences_words):
        ft_matrix = []
        for w in sw:
            ft_matrix.append(ft_model[w])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [11]:
wordtoken_features = get_wordtoken_fts(tweets, opt_wt_length)

In [12]:
wordtoken_features[0]

[array([-2.23397613e-02, -7.97595177e-03, -5.55484146e-02, -3.11111547e-02,
         8.26205760e-02, -1.19878248e-04, -2.74167769e-02, -2.09974889e-02,
         1.52380913e-01,  1.31636232e-01,  1.82123315e-02,  6.88945428e-02,
         3.55035365e-02,  4.57400978e-02, -5.93751259e-02,  1.66083276e-02,
        -1.62978452e-02,  4.86690514e-02,  4.35634665e-02, -2.05784887e-02,
         4.12283093e-02,  1.01482876e-01,  5.57731017e-02, -1.15701586e-01,
         1.75112544e-03, -1.88791193e-02,  6.89997971e-02, -3.00547313e-02,
        -3.18159536e-02,  1.52013958e-01, -3.12297978e-02,  1.13977613e-02,
        -1.18614405e-01, -8.11770111e-02, -1.14518575e-01, -2.80886143e-02,
         7.45225921e-02, -6.10238165e-02, -1.89037751e-02, -4.55767140e-02,
         2.32991595e-02,  1.13217317e-01,  9.74094793e-02,  4.11681682e-02,
         1.99787505e-02, -6.33342713e-02, -1.42207947e-02,  1.52947754e-02,
        -4.96735871e-02,  4.96025011e-02, -1.27109915e-01,  1.43254995e-01,
        -8.7

### Supervised fastText wordtokens training

In [13]:
if not os.path.exists('hsd/Poleval2019/fasttext_pos.ft'):
    # only if morfeusz2 is installed
    '''with open('hsd/Poleval2019/fasttext_pos.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, pos(t)))'''
    # otherwise load pos strings from outer source
    with open('hsd/Poleval2019/pos_sentences.pkl', 'r') as f:
        pos_sentences = pickle.load(f)
    with open('hsd/Poleval2019/fasttext_pos.ft', 'a') as f:
        for ps, l in list(zip(pos_sentences, labels)):
            f.write('__label__{} {}\n'.format(l, ps))
        

# load fasttext pos model or train & save if none
if os.path.exists('hsd/Poleval2019/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/Poleval2019/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/Poleval2019/fasttext_pos.ft',
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/Poleval2019/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [14]:
def get_pos_fts(data, length):
    
    # only if morfeusz2 is installed
    '''pos_sentences = [pos(sentence) for sentence in tqdm(sentences)]'''
    # otherwise load pos strings from outer source
    with open('hsd/Poleval2019/pos_sentences.pkl', 'r') as f:
        pos_sentences = pickle.load(f)
    
    
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    pos_tags = [pad_words(pt, length) for pt in pos_tags]
    
    ft_matrices = []
    for pt in tqdm(pos_tags):
        ft_matrix = []
        for t in pt:
            ft_matrix.append(ft_pos_model[t])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [15]:
pos_features = get_pos_fts(tweets, opt_pos_length)

In [16]:
pos_features[0]

[array([ 0.11104294, -0.0135534 , -0.08745389, -0.084714  ,  0.17829643,
        -0.05323748, -0.15070857, -0.09474131,  0.27817327,  0.24677292,
        -0.04408586,  0.19261555, -0.04393361,  0.07514106, -0.17158775,
         0.03967829,  0.13769884, -0.08801575,  0.08030478,  0.06578112,
         0.04733757,  0.1021822 ,  0.11170699, -0.19328351, -0.04927928,
        -0.12543932,  0.18877542, -0.11019258,  0.09915238,  0.38049096,
        -0.00230696, -0.06357166, -0.16522805,  0.00417294, -0.2974559 ,
        -0.05214699,  0.16999485,  0.11123104, -0.02429225,  0.0135356 ,
        -0.07478285,  0.08781778,  0.30113164,  0.19161917,  0.0436292 ,
        -0.18411101,  0.14593004,  0.17290261,  0.0197932 ,  0.17641643,
        -0.12125841,  0.18289606, -0.11839268,  0.08221723, -0.15027477,
        -0.07371646, -0.04750825, -0.10986844, -0.09617807,  0.19505647,
         0.15364912,  0.24301688,  0.03421713,  0.13105609, -0.0576263 ,
         0.02631858, -0.33105597,  0.08890928,  0.0

In [17]:
np.array(wordtoken_features).shape

(11041, 13, 200)

In [18]:
np.array(pos_features).shape

(11041, 27, 200)

### All features

In [19]:
#Now join them all up
features = np.concatenate([wordtoken_features, pos_features],axis=1)

In [20]:
features.shape

(11041, 40, 200)

In [21]:
features[0]

array([[-2.2339761e-02, -7.9759518e-03, -5.5548415e-02, ...,
         3.5573591e-02, -3.6786549e-02, -7.0850581e-02],
       [-1.8802367e-02, -7.5335354e-03, -6.3467592e-02, ...,
         5.0666522e-02, -5.2971963e-02, -9.6865162e-02],
       [-2.3196351e-03, -4.9491697e-03, -3.6272628e-05, ...,
        -1.2192096e-03,  2.3642126e-03,  2.0747504e-04],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

## Save features & labels

In [22]:
archive = dir_archive('hsd/Poleval2019/X_y_{}'.format(MODEL), {'features': features, 'labels': labels,
                                                                'wt_num': np.array(wordtoken_features).shape[1]}, serialized=True)
archive.dump()
del archive